<left>
    <img src="https://kpi.ua/files/images/kpi.png" width="300" alt="kpi logo"  />
</left>

# **Курсова робота з дисципліни  "Методи оптимізації"**  
## **Тема : Метод найменших квадратів**
*Студентa групи КМ-81 

Ханіна Данила Андрійовича*

**Дана курсова робота направлена на дослідження збіжності методу найменших квадратів.**

***Постановка задачі***

Дослідити збіжність методу найменших квадратів Гауса-Ньютона при мінімізації функції Розенброка  в залежності від:
1.	Величини кроку h при обчисленні похідних.
2.	Модифікації Левенберга-Марквардта.
3.	Схеми обчислення похідних.
4.	Вигляду критерію закінчення.
  
Використати метод штрафних функцій (метод внутрішньої точки) для умовної оптимізації в залежності від:
1.	Розташування локального мінімума (всередині/поза допустимою областю).
2.	Виду допустимої області (випукла/невипукла).


## Зміст

* Анотація 
* Вступ 
* Вирішення поставленої задачі
  * Імпортування бібліотек
  * Визначення констант
  * Опис функцій
  * Обчислення
  * Відображення результатів  
* Висновки 
* Про виконавця
* Посилання

## Анотація

**Головне завдання -** є дослідження збіжності методу найменших квадратів,  в залежності від певних умов, таких як величина кроку h при обчисленні похідних, модифікація Левенберга-Марквардта, вибір схеми обчислення похідних та вигляд критерію закінчення.  

**Мета -**  швидка збіжність методу та висока точність знайденої точки мінімума, аналіз методу.  

**Складнощі,під час виконання -** підбір параметрів у методах вручну,реалізації усіх компонентів досліджуваних методів чисельно.


## Вступ

***В даній курсовій роботі необхідно дослідити збіжність методу найменших квадратів Гауса-Ньютона при мінімізації функції Розенброка та  застосувати метод внутрішньої точки для умовної оптимізації.***   
  
    
Функція Розенброка:
$$ f(\overline x) =  100(x_1^2-x_2)^2 + (x_1-1)^2  $$  
Дана функція має мінімум в точці $\overline x = 
\begin{pmatrix}
  1 \\
  1
 \end{pmatrix}$ в якій значення функції $ f(\overline x) = 0 $.



---
***Теоретичні відомості :***



*Метод найменших квадратів Гауса-Ньютона* - це ітераційний чисельний метод знаходження рішення задачі найменших квадратів. Є різновидом методу Ньютона.
Відміністю є те ,що алгоритм Гауса-Ньютона може бути використаний тільки для мінімізації суми квадратів: 
$$ F(x) = \displaystyle\sum_{i=1}^{m} (f_i(x))^2 = \|f(x)\|^2 $$  

*Переваги методу:* не вимагає обчислення других похідних, а також обчислення та обернення матриці Гессе на кожній ітерації, але при цьому швидкість збіжності методу найменших квадратів Гауса-Ньютона також може досягати квадратичної; простий в реалізації.

*Недолік методу:*  якщо член другого порядку Q завеликий це призводить до некоректної роботи. 

*Додаткова умова* для можливості застосовування методу -  кількість змінних $\overline x = (x_1, x_2, ..., x_n)$ не перевищує кількість функцій $f = (f_1, f_2, ..., f_m)$, тобто $ n \geq m $.  

Нехай у нас є початкова точка $x^{(0)}$ , тоді наближення до точки мінімума виконується на кожній наступній ітерації за наступною формулою: $$ x^{(k+1)} = x^{(k)} - (J^T(x^{(k)}) J(x^{(k)}))^{-1}J^T(x^{(k)})f(x^{(k)}) $$

*Метод внутрішньої точки* - різновид методу штрафних функцій, що  задачу умовної оптимізації зводить  до вирішення послідовності задач безумовної оптимізації.  

Такі задачі будуть набувати наступного загального вигляду:
$$ P(x,R) = f(x) + F(R,g(x),h(x)) $$ де штрафний параметр $ R \longrightarrow 0 $.  


## Вирішення поставленої задачі

### Імпортування бібліотек

In [1]:
import numpy as np
from math import sqrt
import plotly.graph_objects as go
import matplotlib.pyplot as plt

### Визначення констант

In [2]:

x0 = np.array([-1.2, 0.0]) 

h1 = 0.001  
h2 = 0.0001
h3 = 0.000001

itr = 500  

eps1 = 0.001  
eps2 = 0.0001
eps_lst = [0.1, 0.01, 0.001, 0.0001, 0.00001] 

lambda_fix = 1000  

Jr = np.zeros((2,2), dtype=float)  
Jc = np.zeros((2,2), dtype=float)  

x0_2 = np.array([-0.5,0.5])  
Jrp = np.zeros((3,2), dtype=float)  

restr1 = 0  
restr2 = 1  
restr3 = 2  
restr4 = 3  

R = 1  
R2 = 10   

eps3 = 0.01

### Опис функцій

#### Опис функцій,які використовуються для вирішення задачі безумовної оптимізації

Дана функція *rosenbrock()* повертає значення функції Розенброка в точці х, яка приймається у форматі numpy масиву як вхідний параметр:

In [3]:
def rosenbrock(x):
    return ( 100*((x[0]**2-x[1])**2)+(x[0]-1)**2 )

Наступні три функції необхідні нам для обчислення похідних. Похідні будемо обчислювати за правою та центральною схемами.  
*Чисельне обчислення похідних за правою схемою*: $$f'_{r} = \frac{f(x+h) - f(x)}{h}$$  
*Чисельне обчислення похідних за центральною схемою*: $$f'_{m} = \frac{f(x+h) - f(x-h)}{2h}$$

Функція rosenbrock_elements() також приймає на вхід поточну точку х і розкладає функцію Розенброка на окремі функції:

In [4]:
def rosenbrock_elements(x):
    return  (x[0]-1)**2,100*((x[0]**2-x[1])**2),

Функції J_rigth() та J_сenter() приймають в якості параметрів точку х та довжину кроку h і повертають значення Якобіану:

In [5]:
#Якобіан. Права схема
def J_rigth(x, h):
    for i in range(2):
        for j in range(2):
            Jr[i][j] = (rosenbrock_elements((x[0]+((j+3)%2)*h, x[1]+((j+2)%2)*h))[i] - rosenbrock_elements(x)[i])/h
    return Jr


#Якобіан.Центральна схема
def J_center(x, h):
    for i in range(2):
        for j in range(2):
            Jc[i][j] = (rosenbrock_elements((x[0]+((j+3)%2)*h, x[1]+((j+2)%2)*h))[i] -
                        rosenbrock_elements((x[0]-((j+3)%2)*h, x[1]-((j+2)%2)*h))[i])/(2*h)
    return Jc


Функція norma() приймає в якості параметру точку х та повертає значення її норми:

In [6]:
def norma(x):
    return sqrt(x[0]**2 + x[1]**2)

В даній курсовій роботі критерій закінчення буде мати два вигляда.  
*Перший варіант* критерію закінчення :
$$ \|\nabla f(x^{(k)})\| \leq \varepsilon $$  
*Другий варіант* критерію закінчення :  
$$  \begin{cases}
    \frac{\|x^{(k-1)} - x^{(k)}\|}{\|x^{(k)}\|} \leq \varepsilon \\     
    \frac{|f(x^{(k-1)}) - f(x^{(k)})|}{|f(x^{(k)})|} \leq \varepsilon
  \end{cases} $$

Функція end1() приймає поточну точку х, крок h та точність обчислення. Функція end2() приймає поточну і попередню точки х, значення функції в цих точках та точність обчислення.  


In [7]:

def end1(*args):
    point, h, eps = args
    grad = np.zeros((2))
    grad[0] = (rosenbrock((point[0]+h, point[1])) - rosenbrock((point[0]-h, point[1])))/(2*h)
    grad[1] = (rosenbrock((point[0], point[1]+h)) - rosenbrock((point[0], point[1]-h)))/(2*h)
    if norma(grad) <= eps:
        print(f'Похибка: {norma(grad)}')
        return True
    else:
        print(f'Похибка: {norma(grad)}')
        return False

def end2(*args):
    p1,p2,Fp1,Fp2,eps = args
    term1 = norma(p2-p1)/norma(p1)
    term2 = abs(Fp2-Fp1)/abs(Fp1)
    if (term1<=eps) and (term2<=eps):
        # print(f'Похибка_1: {term1}')
        # print(f'Похибка_2: {term2}')
        return True
    else:
        # print(f'Похибка_1: {term1}')
        # print(f'Похибка_2: {term2}')
        return False

Функція find_min() є реалізацією методу найменших квадратів Гауса-Ньютона.  
Дана функція приймає такі *вхідні параметри*: початкова точка х, якобіан, критерий закінчення, крок h для обчислення якобіану, точність обчислення  ε та максимальну кількість ітерацій.  
Дана функція знаходить наступні : кожну наступну точку, перевіряє виконання критерію закінчення, поки не буде досягнуто бажаної точності або поки не будуть пройдені всі ітерації.

In [8]:
def find_min(x, J, criteria, krok, eps, itr):
    it = 0
    prevX = x
    prevF = rosenbrock(x)
    while it < itr:
        Jacob = J(prevX, krok)
        Jtr = Jacob.T
        term1 = np.linalg.pinv(Jtr.dot(Jacob))
        term2 = Jtr.dot((np.array(rosenbrock_elements(prevX)).reshape(2,1)))
        nextX = prevX.reshape(2,1) - term1.dot(term2)
        nextX = nextX.reshape(2)
        nextF = rosenbrock(nextX)
        if criteria == end2:
            if end2(prevX,nextX,prevF,nextF,eps):
                return nextX, nextF, it
        else:
            if  end1(nextX, krok, eps):
                return nextX, nextF, it

        prevX = nextX[:]
        prevF = nextF
        it += 1

    return nextX, nextF, it

Функція find_min_lev_marq() є модифікацією Левенберга-Марквардта. 

Дана функція знаходить наступні : кожну наступну точку, перевіряє виконання критерію закінчення, поки не буде досягнуто бажаної точності або поки не будуть пройдені всі ітерації.

In [9]:
def find_min_lev_marq(x, J, criteria, krok, eps, lm, itr):
    it = 0
    lm_next = lm
    prevX = x
    prevF = rosenbrock(x)
    while it < itr:
        Jacob = J(prevX, krok)
        Jtr = Jacob.T
        term0 = Jtr.dot(Jacob) + lm_next*np.eye(2)
        term1 = np.linalg.pinv(term0)
        term2 = Jtr.dot((np.array(rosenbrock_elements(prevX)).reshape(2,1)))
        nextX = prevX.reshape(2,1) - term1.dot(term2)
        nextX = nextX.reshape(2)
        nextF = rosenbrock(nextX)
        if criteria == end2:
            if end2(prevX,nextX,prevF,nextF,eps):
                return nextX, nextF, it
        else:
            if end1(nextX, krok, eps):
                return nextX, nextF, it
        if (nextF-prevF) > 0:
            lm_next = lm_next * 100
        else:
            lm_next = lm_next / 100
        prevX = nextX[:]
        prevF = nextF
        it += 1
    return nextX, nextF, it

Функція penalty() приймає значення точки х і повертає значення функцій-обмежень в точці.

Функції rosenbrock_penalty() і rosenbrock_penalty_elem() приймають на вхід в якості параметрів точку х, штрафний параметр R, номер обмеження із сукупності обмежень, при яких досліджується метод.

Функція rosenbrock_penalty() повертає значення цільової функції задачі умовної оптимізації в точці,а функція rosenbrock_penalty_elem() повертає значення функцій, які представляють собою розкладення цільової функції задачі умовної оптимізації, в даній точці.

In [10]:
def penalty(x):
    return [-x[0]**2 - x[1]**2 + 4, x[0]**2 + x[1]**2 - 1, (-x[0]**2 - x[1]**2 + 4) + (x[0]**2 + x[1]**2 - 1), (-x[0]**2 - x[1]**2 + 1) + (x[0]**2 + x[1]**2 - 0.25)]

def rosenbrock_penalty(x, r, ind):
    return ((1-x[0])**2 + 100*(x[1] - x[0]**2)*(x[1] - x[0]**2) + r*(penalty(x)[ind]))

def rosenbrock_penalty_elem(x, r, ind):
    return ((1-x[0])**2, 100*(x[1] - x[0]**2)*(x[1] - x[0]**2), r*(penalty(x)[ind]))

Функція J_rigth_penalty() приймає в якості параметрів точку х, крок h для обчислення похідних, штрафний параметр R, номер обмеження із сукупності обмежень, при яких досліджується метод, та повертає якобіан, компоненти якого знаходяться шляхом застосування правої схеми обчислення похідних:

In [11]:
def J_rigth_penalty(x, h, r, ind):
    for i in range(3):
        for j in range(2):
            Jrp[i][j] = (rosenbrock_penalty_elem((x[0]+((j+3)%2)*h, x[1]+((j+2)%2)*h), r, ind)[i]
                        - rosenbrock_penalty_elem(x, r, ind)[i])/h
    return Jrp

Функція find_min_penalty() приймає наступні вхідні параметри: початкова точка х, крок h для обчислення похідних, точність обчислення  ε , штрафний параметр R, номер обмеження із сукупності обмежень, при яких досліджується метод, та максимальна кількість ітерацій у ході пошуку мінімальної точки, при якій спрацює зупинка пошуку мінімуму у разі не досягнення заданої точності. 

In [12]:
def find_min_penalty(x, krok, eps, r, ind, itr):
    it = 0
    prevX = x
    if penalty(prevX)[ind] >= 0:
        prevF = rosenbrock(x)
    else:
        prevF = rosenbrock_penalty(prevX, r, ind)
    while it < itr:
        if penalty(prevX)[ind] >= 0:
            Jacob = J_rigth(prevX, krok)
            Jtr = Jacob.T
            term1 = np.linalg.pinv(Jtr.dot(Jacob))
            term2 = Jtr.dot((np.array(rosenbrock_elements(prevX)).reshape(2, 1)))
            nextX = prevX.reshape(2, 1) - term1.dot(term2)
            nextX = nextX.reshape(2)
            nextF = rosenbrock(nextX)
            if end2(prevX, nextX, prevF, nextF, eps):
                return nextX, nextF, it
        else:
            Jacob = J_rigth_penalty(prevX, krok, r, ind)
            Jtr = Jacob.T
            term1 = np.linalg.pinv(Jtr.dot(Jacob))
            term2 = Jtr.dot((np.array(rosenbrock_penalty_elem(prevX, r, ind)).reshape(3,1)))
            nextX = prevX.reshape(2,1) - term1.dot(term2)
            nextX = nextX.reshape(2)
            nextF = rosenbrock_penalty(nextX, r, ind)
            if end2(prevX,nextX,prevF,nextF,eps):
                return nextX, nextF, it

        prevX = nextX[:]
        prevF = nextF
        r *= 10
        it += 1

    return nextX, nextF, it

### Обчислення

#### Обчислення для вирішення задачі безумовної оптимізації

***Дослідження збіжності методу найменших квадратів Гауса-Ньютона при мінімізації функції Розенброка в залежності від величини кроку h при обчисленні похідних.***

In [13]:
x_min_1, f_1, itr_1 = find_min(x0, J_rigth, end1, h1, eps1, itr) 
x_min_2, f_2, itr_2 = find_min(x0, J_rigth, end1, h2, eps1, itr) 
x_min_3, f_3, itr_3 = find_min(x0, J_rigth, end1, h3, eps1, itr) 

Похибка: 393.61796894055675
Похибка: 339.98948797885515
Похибка: 249.01311165313982
Похибка: 148.2133862656542
Похибка: 80.25597199614067
Похибка: 41.463832505758106
Похибка: 20.809772273819615
Похибка: 10.155977347147804
Похибка: 4.734460362563161
Похибка: 1.969823733506748
Похибка: 0.462905399997339
Похибка: 3.7500993186217952
Похибка: 1.5385986299837304
Похибка: 0.057885812816785266
Похибка: 0.67191972454204
Похибка: 0.532904199106862
Похибка: 0.4279437067560195
Похибка: 0.35051086500536355
Похибка: 0.2930979936520913
Похибка: 0.24982479889183667
Похибка: 0.21654035173890168
Похибка: 0.19040506196369572
Похибка: 0.16948098580989604
Похибка: 0.15243131725921116
Похибка: 0.1383186664193704
Похибка: 0.12647346452173533
Похибка: 0.11640841865949642
Похибка: 0.10776244576575332
Похибка: 0.10026342023759209
Похибка: 0.09370301996346463
Похибка: 0.08791945249924452
Похибка: 0.08278539227736029
Похибка: 0.07819941945340107
Похибка: 0.07407984986398984
Похибка: 0.07036022348849497
Похибка: 0

Похибка: 0.003632700249425415
Похибка: 0.003539086442465874
Похибка: 0.003450113846131358
Похибка: 0.00336544828633798
Похибка: 0.00328478674734386
Похибка: 0.003207853837246565
Похибка: 0.0031343987223921904
Похибка: 0.003064192459408109
Похибка: 0.002997025667801205
Похибка: 0.0029327064914562747
Похибка: 0.002871058809851701
Похибка: 0.002811920663095224
Похибка: 0.0027551428619537917
Похибка: 0.002700587757408091
Похибка: 0.0026481281508082023
Похибка: 0.002597646323614456
Похибка: 0.002549033173206404
Похибка: 0.0025021874415409127
Похибка: 0.002457015025497405
Похибка: 0.002413428357073237
Похибка: 0.002371345848517938
Похибка: 0.002330691391176494
Похибка: 0.002291393905407825
Похибка: 0.002253386932546385
Похибка: 0.0022166082671773724
Похибка: 0.002180999622935489
Похибка: 0.0021465063296377115
Похибка: 0.0021130770580280642
Похибка: 0.002080663568762047
Похибка: 0.0020492204851036235
Похибка: 0.0020187050829700483
Похибка: 0.0019890771014109054
Похибка: 0.0019602985675594
Пох

***Дослідження збіжності методу найменших квадратів Гауса-Ньютона при мінімізації функції Розенброка в залежності від модифікації Левенберга-Марквардта***

In [14]:
x_min_lev1, f_lev1, it1 = find_min_lev_marq(x0, J_rigth, end1, h1, eps1, lambda_fix, itr)
x_min_lev2, f_lev2, it2 = find_min_lev_marq(x0, J_rigth, end1, h2, eps1, lambda_fix, itr)
x_min_lev3, f_lev3, it3 = find_min_lev_marq(x0, J_rigth, end1, h3, eps1, lambda_fix, itr)

Похибка: 339.1577837901261
Похибка: 162.89870494717255
Похибка: 184.30585665557166
Похибка: 89.79269700491966
Похибка: 113.63725680048397
Похибка: 87.78316284227517
Похибка: 52.943920940803956
Похибка: 28.481027677496193
Похибка: 14.285185382472921
Похибка: 6.644580335719959
Похибка: 2.6212543573277998
Похибка: 0.34295150118568923
Похибка: 8.612665730921659
Похибка: 4.348570898021436
Похибка: 2.2848380345066954
Похибка: 1.2570058301053562
Похибка: 0.7668066517652489
Похибка: 0.6996761902511198
Похибка: 0.32959501835300875
Похибка: 0.42775269271370114
Похибка: 0.18660360789577948
Похибка: 0.1263343042608172
Похибка: 0.16332302123096307
Похибка: 0.16932756948518887
Похибка: 0.16359774683894374
Похибка: 0.1537382329761847
Похибка: 0.1429121323569664
Похибка: 0.13241609589215644
Похибка: 0.12273504813555228
Похибка: 0.11399953104848584
Похибка: 0.10618976382092356
Похибка: 0.09922774826761506
Похибка: 0.09301859288273732
Похибка: 0.08746844949458334
Похибка: 0.08249163099977258
Похибка: 0.

Похибка: 0.0029830158454311174
Похибка: 0.0029747716133848357
Похибка: 0.0029665681432065596
Похибка: 0.0029584051341240186
Похибка: 0.002950282288631635
Похибка: 0.0029421993118657823
Похибка: 0.00293415591187734
Похибка: 0.0029261517995931427
Похибка: 0.002918186689014459
Похибка: 0.0029102602963429866
Похибка: 0.0029023723411144573
Похибка: 0.0028945225450459763
Похибка: 0.0028867106329097205
Похибка: 0.002878936331931766
Похибка: 0.002871199372140287
Похибка: 0.002863499485927817
Похибка: 0.002855836408255438
Похибка: 0.0028482098767236216
Похибка: 0.0028406196314024338
Похибка: 0.0028330654148703413
Похибка: 0.0028255469718708585
Похибка: 0.002818064049748575
Похибка: 0.0028106163982606815
Похибка: 0.002803203769476839
Похибка: 0.0027958259177346483
Похибка: 0.002788482599601635
Похибка: 0.0027811735740607274
Похибка: 0.002773898602279843
Похибка: 0.002766657447627415
Похибка: 0.002759449875692544
Похибка: 0.002752275654028136
Похибка: 0.0027451345525484603
Похибка: 0.002738026343

***Дослідження збіжності методу найменших квадратів Гауса-Ньютона при мінімізації функції Розенброка в залежності від схеми обчислення похідних***

In [15]:
x_min_c1, f_c1, it_c1 = find_min(x0, J_center, end1, h3, eps1, itr)
x_min_c2, f_c2, it_c2 = find_min(x0, J_rigth,  end1, h3, eps1, itr)

Похибка: 394.08166660025523
Похибка: 340.3144905256471
Похибка: 249.4448467973976
Похибка: 148.74363727275605
Похибка: 80.8541793894462
Похибка: 42.105317454808436
Похибка: 21.47931785784048
Похибка: 10.847205657608473
Похибка: 5.45059940750849
Похибка: 2.7320625641059646
Похибка: 1.3677237103703748
Похибка: 0.684285176330058
Похибка: 0.3422484452326735
Похибка: 0.17115069025629792
Похибка: 0.0855819624637712
Похибка: 0.04279263563601141
Похибка: 0.021396731441660443
Похибка: 0.010698469144599883
Похибка: 0.005349260444864418
Похибка: 0.002674636707079827
Похибка: 0.0013373199915442334
Похибка: 0.0006686604216867543
Похибка: 394.08120268024044
Похибка: 340.31416546804326
Похибка: 249.44441529385696
Похибка: 148.7431078308412
Похибка: 80.85358300438429
Похибка: 42.104679684676626
Похибка: 21.478655820818112
Похибка: 10.846529822516388
Похибка: 5.449915869001681
Похибка: 2.731374768634157
Похибка: 1.3670335540147969
Похибка: 0.6835936584166982
Похибка: 0.34155602446419153
Похибка: 0.1704

***Дослідження збіжності методу найменших квадратів Гауса-Ньютона при мінімізації функції Розенброка в залежності від вигляду критерію закінчення***

In [16]:
xmin_cr1, f_cr1, it_cr1 = find_min(x0, J_center, end1, h3, eps2, itr)
xmin_cr2, f_cr2, it_cr2 = find_min(x0, J_center, end2, h3, eps2, itr)

Похибка: 394.08166660025523
Похибка: 340.3144905256471
Похибка: 249.4448467973976
Похибка: 148.74363727275605
Похибка: 80.8541793894462
Похибка: 42.105317454808436
Похибка: 21.47931785784048
Похибка: 10.847205657608473
Похибка: 5.45059940750849
Похибка: 2.7320625641059646
Похибка: 1.3677237103703748
Похибка: 0.684285176330058
Похибка: 0.3422484452326735
Похибка: 0.17115069025629792
Похибка: 0.0855819624637712
Похибка: 0.04279263563601141
Похибка: 0.021396731441660443
Похибка: 0.010698469144599883
Похибка: 0.005349260444864418
Похибка: 0.002674636707079827
Похибка: 0.0013373199915442334
Похибка: 0.0006686604216867543
Похибка: 0.0003343303340271115
Похибка: 0.00016716521450223657
Похибка: 8.358263565905007e-05


#### Обчислення для вирішення задачі умовної оптимізації

***Метод внутрішньої точк для умовної оптимізації в залежності від розташування локального мінімума (всередині допустимої випуклої області)***

In [17]:
xmin1, f1, it1 = find_min_penalty(x0_2, h1, eps3, R, restr1, itr)
xmin1_2, f1_2, it1_2 = find_min_penalty(x0_2, h1, eps3, R2, restr1, itr)

***Метод внутрішньої точк для умовної оптимізації в залежності від розташування локального мінімума (поза допустимою випуклою областю)***

In [18]:
xmin2, f2, it2 = find_min_penalty(x0_2, h1, eps3, R, restr2, itr)
xmin2_2, f2_2, it2_2 = find_min_penalty(x0_2, h1, eps3, R2, restr2, itr)

***Метод внутрішньої точки для умовної оптимізації в залежності від розташування локального мінімума (всередині допустимої невипуклої області)***

In [19]:
xmin3, f3, it3 = find_min_penalty(x0_2, h1, eps3, R, restr3, itr)
xmin3_2, f3_2, it3_2 = find_min_penalty(x0_2, h1, eps3, R2, restr3, itr)

***Метод зовнішньої точки для умовної оптимізації в залежності від розташування локального мінімума (поза допустимою випуклою областю)***

In [20]:
xmin4, f4, it4 = find_min_penalty(x0_2, h1, eps3, R, restr4, itr)
xmin4_2, f4_2, it4_2 = find_min_penalty(x0_2, h1, eps3, R2, restr4, itr)

### Результати

#### Результати вирішення задачі безумовної оптимізації

**Результати метода наименьших квадратов Гауса-Ньютона, при різних значеннях h:**

Обчислення проводились з точністю $\varepsilon$ = 0.001 , похідні обчислювались за правою схемою, було обрано перший варіант критерію закінчення(значення норми градієнта функції в точці).

In [21]:
print(f'При h={h1} мінімум функції f(x_min) = {f_1}  в точці x_min = {x_min_1} на {itr_1} ітерації.')
print(f'При h={h2} мінімум функції f(x_min) = {f_2}  в точці x_min = {x_min_2} на {itr_2} ітерації.')
print(f'При h={h3} мінімум функції f(x_min) = {f_3}  в точці x_min = {x_min_3} на {itr_3} ітерації.')

При h=0.001 мінімум функції f(x_min) = 3.901414978304093e-09  в точці x_min = [1.00000208 1.0000104 ] на 500 ітерації.
При h=0.0001 мінімум функції f(x_min) = 5.051829429740363e-10  в точці x_min = [1.00000075 1.00000374] на 149 ітерації.
При h=1e-06 мінімум функції f(x_min) = 8.835178230351024e-11  в точці x_min = [0.9999996  0.99999825] на 20 ітерації.


**Результати метода наименьших квадратов Гауса-Ньютона, при різних схемах обчислення похідних**

Обчислення проводились з точністю $\varepsilon$ = 0.001 , h = 0,000001, було обрано перший варіант критерію закінчення(значення норми градієнта функції в точці).

In [22]:
print(f'При обчисленні похідних за ЦЕНТРАЛЬНОЮ схемою, при h={h3} мінімум функції f(x_min) = {f_c1}  в точці x_min = {x_min_c1} на {it_c1} ітерації.')
print(f'При обчисленні похідних за ПРАВОЮ схемою,      при h={h3} мінімум функції f(x_min) = {f_c2}   в точці x_min = {x_min_c2} на {it_c2} ітерації.')

При обчисленні похідних за ЦЕНТРАЛЬНОЮ схемою, при h=1e-06 мінімум функції f(x_min) = 2.2445617890707633e-10  в точці x_min = [0.99999948 0.99999745] на 21 ітерації.
При обчисленні похідних за ПРАВОЮ схемою,      при h=1e-06 мінімум функції f(x_min) = 8.835178230351024e-11   в точці x_min = [0.9999996  0.99999825] на 20 ітерації.


**Результати метода наименьших квадратов Гауса-Ньютона, при різних виглядах критерію закінчення**

Обчислення проводились з точністю $\varepsilon$ = 0.0001 , похідні обчислювались за центральною схемою, h=0,000001.

In [23]:
print(f'При критерій закінчення по відносній похибці,       при h={h3} мінімум функції f(x_min) = {f_cr1}  в точці x_min = {xmin_cr1} на {it_cr1} ітерації.')
print(f'При критерію закінчення по значенню норми градієнта, при h={h3} мінімум функції f(x_min) = {f_cr2}  в точці x_min = {xmin_cr2} на {it_cr2} ітерації.')

При критерій закінчення по відносній похибці,       при h=1e-06 мінімум функції f(x_min) = 3.5071059788016414e-12  в точці x_min = [0.99999993 0.99999968] на 24 ітерації.
При критерію закінчення по значенню норми градієнта, при h=1e-06 мінімум функції f(x_min) = 1.282396939430565e-26  в точці x_min = [1. 1.] на 53 ітерації.


#### Результати вирішення задачі умовної оптимізації

В усіх  випадках буде застосовано праву схему обчислення похідних з точністю ε  = 0.01.

***Результати при розташуванні локального мінімума всередині допустимої випуклої області***

*Область*: $x_1^2 + x_2^2 \leq 9$  
*Початкова точка*: $ x^{(0)} = 
\begin{pmatrix}
  -0,5 \\
  0,5
 \end{pmatrix}$

![Результати при розташуванні локального мінімума всередині допустимої випуклої області](1.png)

In [24]:
print(f'При h={h1}, R=1 мінімум функції f(x_min) = {f1}  в точці x_min = {xmin1} на {it1} ітерації.')
print(f'При h={h1}, R=10 мінімум функції f(x_min) = {f1_2}  в точці x_min = {xmin1_2} на {it1_2} ітерації.')

При h=0.001, R=1 мінімум функції f(x_min) = 2.2755756748801375e-08  в точці x_min = [1.00000502 1.00002513] на 205 ітерації.
При h=0.001, R=10 мінімум функції f(x_min) = 2.2755756748801375e-08  в точці x_min = [1.00000502 1.00002513] на 205 ітерації.


Результати при розташуванні локального мінімума поза допустимою випуклою областю

*Область*: $x_1^2 + x_2^2 \leq 2$  
*Початкова точка*: $ x^{(0)} = 
\begin{pmatrix}
  -0,5 \\
  0,5
 \end{pmatrix}$

![Результати при розташуванні локального мінімума поза допустимою випуклою областю](2.png)

In [25]:
print(f'При h={h1}, при R=1 мінімум функції f(x_min) = {f2}  в точці x_min = {xmin2} на {it2} ітерації.')
print(f'При h={h1}, при R=10 мінімум функції f(x_min) = {f2_2}  в точці x_min = {xmin2_2} на {it2_2} ітерації.')

При h=0.001, при R=1 мінімум функції f(x_min) = 2.2812690472581354e-08  в точці x_min = [1.00000503 1.00002516] на 219 ітерації.
При h=0.001, при R=10 мінімум функції f(x_min) = 2.215722286392028e-08  в точці x_min = [1.00000494 1.00002475] на 171 ітерації.


***Результати при розташуванні локального мінімума всередині допустимої невипуклої області***

*Область*: $\begin{cases}
    x_1^2 + x_2^2 \geq 2\\
    x_1^2 + x_2^2 \leq 9
  \end{cases}$  
*Початкова точка*: $ x^{(0)} = 
\begin{pmatrix}
  -0.5 \\
  0.5
 \end{pmatrix}$

![Результати при розташуванні локального мінімума всередині допустимої невипуклої області](3.png)

In [26]:
print(f'При h={h1}, при R=1 мінімум функції f(x_min) = {f3}  в точці x_min = {xmin3} на {it3} ітерації.')
print(f'При h={h1}, при R=10 мінімум функції f(x_min) = {f3_2}  в точці x_min = {xmin3_2} на {it3_2} ітерації.')

При h=0.001, при R=1 мінімум функції f(x_min) = 2.2755756748801375e-08  в точці x_min = [1.00000502 1.00002513] на 205 ітерації.
При h=0.001, при R=10 мінімум функції f(x_min) = 2.2755756748801375e-08  в точці x_min = [1.00000502 1.00002513] на 205 ітерації.


***Результати при розташуванні локального мінімума поза допустимою невипуклою областю***

*Область*: $\begin{cases}
    x_1^2 + x_2^2 \leq 1\\
    x_1^2 + x_2^2 \geq 0.25
  \end{cases}$  
  *Початкова точка*: $ x^{(0)} = 
\begin{pmatrix}
  -0,5 \\
  0,5
 \end{pmatrix}$

![Результати при розташуванні локального мінімума поза допустимою невипуклою областю](4.png)

In [27]:
print(f'При h={h1}, при R=1 мінімум функції f(x_min) = {f4}  в точці x_min = {xmin4} на {it4} ітерації.')
print(f'При h={h1}, при R=10 мінімум функції f(x_min) = {f4_2}  в точці x_min = {xmin4_2} на {it4_2} ітерації.')

При h=0.001, при R=1 мінімум функції f(x_min) = 2.2755756748801375e-08  в точці x_min = [1.00000502 1.00002513] на 205 ітерації.
При h=0.001, при R=10 мінімум функції f(x_min) = 2.2755756748801375e-08  в точці x_min = [1.00000502 1.00002513] на 205 ітерації.


### Висновки

***Досліджуючи метод наименьших квадратов в залежності від значення кроку h***, який використовуєтся у обчисленні похідних, *було виявлено*, що *при зменшенні значення кроку h збіжність методу прискорюється*.   
За умови однакової точності та однакової схеми обчислення похідних(в даному випадку правої схеми),  при h=0.0001 точку мінімума було знайдено на 149 ітерації,а при h=0.00001 було досягнуто точки мінімума на 20 ітерації.

***Досліджуючи метод наименьших квадратов в залежності від застосування модифікації Левенберга-Марквардта***, то за умови обрання тих самих параметрів (точність  ε , крок h, схема обчислення похідних), що і у попередньому варіанті, отримали нижчу швидкість збіжності, ніж в МНК Гауса-Ньютона. Але також слід відмітити, що при використані h=0.001 кількість ітерації у модифікації Левенберга-Марквардта була менша за кількість ітерації у метода наименьших квадратов Гауса-Ньютона.

***Досліджуючи метода наименьших квадратов в залежності від схеми обчислення похідних***. було виявлено ,що *за використання правої схеми обчислення похідних МНК Гауса-Ньютона збігається до точки мінімума швидше, ніж за використання центральної схеми обчислення похідних*.

***Досліджуючи МНК в залежності від вигляду критерію закінчення***, було виявлено ,що при використанні критерію закінчення №1((значення норми градієнта функції в точці) метод досягає точки мінімума значно швидше(за 24 ітерації,в той час коли при критерії №2 за 53 ітерації)

***Отже,*** метод найменших квадратів Гауса-Ньютона володіє дуже високою швидкістю збіжності.

***При використанні методу штрафних функцій(методу внутрішньої точки)*** було виявлено , що :

1.   В невипуклих областях були отримані однакові точки за однакову кількість ітерацій.
2.   При розташуванні поза допустимою випуклою областю було отримано майже таку ж точку як в невипуклих.

Швидкість збіжності у метожі внутрішньої точки приблизно така ж як і при використанні модифікації Левенберга-Марквардта , але менше ніж при МНК Гауса-Ньютона.

## Про виконавця

Студента групи КМ-81,ФПМ  
Ханін Данило 

telegram : @Wa1rus  
github   : https://github.com/Wa11rus

## Посилання

1.   https://habr.com/ru/post/470181/  
2.   https://ru.wikipedia.org/wiki/%D0%90%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC_%D0%9B%D0%B5%D0%B2%D0%B5%D0%BD%D0%B1%D0%B5%D1%80%D0%B3%D0%B0_%E2%80%94_%D0%9C%D0%B0%D1%80%D0%BA%D0%B2%D0%B0%D1%80%D0%B4%D1%82%D0%B0  
3.  https://hub.exponenta.ru/post/postanovka-zadachi-optimizatsii-i-chislennye-metody-ee-resheniya356#3_4_1  
4.  https://www.desmos.com